In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('assets/gapminder.csv', thousands=',', index_col='Year')

In [3]:
data.head()

,Country,life,population,income,region
Year,,,,,
1800,Afghanistan,28.211000,3280000.0,603.0,South Asia
1801,Afghanistan,28.200753,NaN,603.0,South Asia
1802,Afghanistan,28.190507,NaN,603.0,South Asia
1803,Afghanistan,28.180260,NaN,603.0,South Asia
1804,Afghanistan,28.170013,NaN,603.0,South Asia


In [11]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
from bokeh.io import show
from bokeh.plotting import figure

In [ ]:
p = figure(
    height=200, x_axis_type='log',
    x_range=(100, 100000), y_range=(0, 100),
)
p.circle(x=data.loc[2010].income, y=data.loc[2010].life, color='pink')
show(p)

In [ ]:
from bokeh.models import NumeralTickFormatter
p = figure(
    height=200, x_axis_type='log',
    x_range=(100, 100000), y_range=(0, 100),
)
p.circle(x=data.loc[2010].income, y=data.loc[2010].life, color='pink')
p.xaxis[0].formatter = NumeralTickFormatter(format="$0,")
show(p)

In [ ]:
from bokeh.models import ColumnDataSource
source = ColumnDataSource(dict(
    x=data.loc[2010].income,
    y=data.loc[2010].life,
    country=data.loc[2010].Country
    ))

In [ ]:
ColumnDataSource(data.loc[2010])

In [35]:
PLOT_OPTS = dict(
    height=400, x_axis_type='log',
    x_range=(100, 100000), y_range=(0, 100),
)

In [15]:
from bokeh.models import HoverTool
hover = HoverTool(tooltips='@country', show_arrow=False)
p = figure(tools=[hover], **PLOT_OPTS)
p.circle(x='x', y='y', size=20, alpha=0.6,  source=source)
show(p)

TypeError: figure() got multiple values for keyword argument 'tools'

In [8]:
from bokeh.models import ColumnDataSource
source = ColumnDataSource(dict(
    x=data.loc[2010].income,
    y=data.loc[2010].life,
    country=data.loc[2010].Country,
    population=data.loc[2010].population,
    region=data.loc[2010].region
))
source.column_names

['region', 'x', 'population', 'y', 'country']

In [30]:
list(data.region.unique())

['South Asia',
 'Europe & Central Asia',
 'Middle East & North Africa',
 'Sub-Saharan Africa',
 'America',
 'East Asia & Pacific']

In [31]:
from bokeh.palettes import Spectral6
Spectral6

['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']

In [48]:
from bokeh.models import LinearInterpolator, CategoricalColorMapper

from ipywidgets import interact

from bokeh.io import push_notebook

def update(year):
    new_data = dict(
        x=data.loc[year].income, 
        y=data.loc[year].life, 
        country=data.loc[year].Country,
        region=data.loc[year].region,
        population=data.loc[year].population,
    )
    source.data = new_data
    p.title.text = str(year)
    push_notebook()

size_mapper = LinearInterpolator(
    x=[data.population.min(), data.population.max()],
    y=[5, 50]
)
color_mapper = CategoricalColorMapper(
    factors=list(data.region.unique()),
    palette=Spectral6,
)

p = figure(
    title=str(2010), toolbar_location='above',
    tools=[HoverTool(tooltips='@country', show_arrow=False)], 
    **PLOT_OPTS)
p.circle(
    x='x', y='y', 
    size={'field': 'population', 'transform': size_mapper}, 
    color={'field': 'region', 'transform': color_mapper},
    alpha=0.6,  
    source=source,
    legend='region'
)
p.legend.border_line_color = None
p.legend.location = (0, -50)
p.right.append(p.legend[0])
show(p, notebook_handle=True)

In [49]:
interact(update, year=(1800, 2014, 1))